In [1]:
%gui qt

from OCC.Display.SimpleGui import init_display
from OCC.Core.gp import gp_Pnt, gp_Dir, gp_Pln, gp_Vec
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Section
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_EDGE, TopAbs_FACE, TopAbs_VERTEX
from OCC.Core.TopoDS import topods
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge, BRepBuilderAPI_MakeWire, BRepBuilderAPI_MakeFace
from OCC.Core.BRepAdaptor import BRepAdaptor_Surface, BRepAdaptor_Curve
from OCC.Core.GeomAbs import GeomAbs_Plane

from utils_geometry import *
from utils_dstv import *
from utils_visualization import *
from utils_classification import *

import numpy as np
import json


# Start display without blocking
display, start_display, add_menu, add_function_to_menu = init_display()

display.EraseAll()
display.View_Iso()
display.FitAll()

pyside6 backend - Qt version 6.8.3


In [2]:
# Load the STEP and extract 'solid'
from OCC.Extend.DataExchange import read_step_file
# shape = read_step_file("../data/0444-1 ANGLED.step")
# shape = read_step_file("../data/ncTest.step")
# shape = read_step_file("../data/TestEA.step")
shape = read_step_file("../data/TestUEA.step")
# shape = read_step_file("../data/TestPFC.step")
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_SOLID
exp = TopExp_Explorer(shape, TopAbs_SOLID)
solid = exp.Current()

In [3]:
# Utilities to compute DSTV origins and axes from an actual section face

from OCC.Core.gp import gp_Vec, gp_Pnt, gp_Dir, gp_Pln
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Section
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeWire, BRepBuilderAPI_MakeFace
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_EDGE, TopAbs_VERTEX
from OCC.Core.TopoDS import topods
from OCC.Core.BRep import BRep_Tool
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop
import numpy as np

def make_section_face(solid, origin, normal):
    plane = gp_Pln(origin, gp_Dir(normal))
    section = BRepAlgoAPI_Section(solid, plane)
    section.ComputePCurveOn1(True)
    section.Approximation(True)
    section.Build()

    wire_builder = BRepBuilderAPI_MakeWire()
    exp = TopExp_Explorer(section.Shape(), TopAbs_EDGE)
    while exp.More():
        wire_builder.Add(topods.Edge(exp.Current()))
        exp.Next()

    return BRepBuilderAPI_MakeFace(wire_builder.Wire()).Face()

def get_section_face_corners(face):
    exp = TopExp_Explorer(face, TopAbs_VERTEX)
    corners = []
    while exp.More():
        v = topods.Vertex(exp.Current())
        pnt = BRep_Tool.Pnt(v)
        corners.append(pnt)
        exp.Next()
    return corners

def order_corners_by_local_axes(corners, xaxis, yaxis):
    # Project points to local X/Y plane and sort
    def project(pnt):
        return np.array([pnt.X(), pnt.Y(), pnt.Z()])

    pts = [project(p) for p in corners]
    origin = np.mean(pts, axis=0)
    x_dir = np.array([xaxis.X(), xaxis.Y(), xaxis.Z()])
    y_dir = np.array([yaxis.X(), yaxis.Y(), yaxis.Z()])

    local_coords = []
    for pt in pts:
        rel = pt - origin
        lx = np.dot(rel, x_dir)
        ly = np.dot(rel, y_dir)
        local_coords.append((lx, ly, pt))

    # Sort by Y then X
    local_coords.sort(key=lambda v: (-v[1], v[0]))  # Top-left, top-right, bottom-left, bottom-right
    return [gp_Pnt(*v[2]) for v in local_coords]

def get_face_area(face):
    props = GProp_GProps()
    brepgprop.SurfaceProperties(face, props)
    return props.Mass()

def compute_dstv_origins_and_axes_from_section(solid, obb_center, he_Z, xaxis, yaxis, zaxis, profile_type):
    
    # Step 1: Try both ends and choose larger
    end1 = gp_Pnt(obb_center.X() + he_Z * zaxis.X(),
                  obb_center.Y() + he_Z * zaxis.Y(),
                  obb_center.Z() + he_Z * zaxis.Z())
    end2 = gp_Pnt(obb_center.X() - he_Z * zaxis.X(),
                  obb_center.Y() - he_Z * zaxis.Y(),
                  obb_center.Z() - he_Z * zaxis.Z())

    face1 = make_section_face(solid, end1, zaxis)
    face2 = make_section_face(solid, end2, zaxis)

    a1 = get_face_area(face1)
    a2 = get_face_area(face2)
    print(f"Face area → end1: {a1:.1f}, end2: {a2:.1f}")

    if a2 > a1:
        print("Swapping Z axis to align with larger end face")
        zaxis = zaxis.Reversed()
        face = face2
        face_origin = end2
    else:
        face = face1
        face_origin = end1

    # Step 2: Get ordered corners and assign DSTV origins
    corners = get_section_face_corners(face)
    if len(corners) < 3:
        print("⚠️ Not enough corners found on section face")
        return {}, {}

    ordered = order_corners_by_local_axes(corners, xaxis, yaxis)
    origin_v = ordered[2]  # bottom-left
    origin_o = ordered[1]  # top-right
    origin_u = ordered[3]  # bottom-right

    origins = {'V': origin_v, 'O': origin_o, 'U': origin_u}

    # Step 3: Flip X axis if angle is unequal and longer leg is not in U
    if profile_type == "L":
        vec_vo = np.array([origin_o.X() - origin_v.X(), origin_o.Y() - origin_v.Y(), origin_o.Z() - origin_v.Z()])
        vec_vu = np.array([origin_u.X() - origin_v.X(), origin_u.Y() - origin_v.Y(), origin_u.Z() - origin_v.Z()])

        len_o = np.linalg.norm(vec_vo)
        len_u = np.linalg.norm(vec_vu)

        if len_o > len_u:
            print("Angle leg in O is longer → swapping origins for U and O")
            origins['U'], origins['O'] = origins['O'], origins['U']
            xaxis = gp_Dir(-xaxis.X(), -xaxis.Y(), -xaxis.Z())

    face_axes = {
        'V': (gp_Vec(-zaxis.X(), -zaxis.Y(), -zaxis.Z()), gp_Vec(yaxis)),
        'O': (gp_Vec(-zaxis.X(), -zaxis.Y(), -zaxis.Z()), gp_Vec(xaxis)),
        'U': (gp_Vec(-zaxis.X(), -zaxis.Y(), -zaxis.Z()), gp_Vec(-xaxis.X(), -xaxis.Y(), -xaxis.Z()))
    }

    return origins, face_axes, face


In [4]:

def fingerprint_shape(solid, obb_dims):
    """
    Analyze face normals and areas to guess profile type: Beam, Channel, Angle, Unknown
    """
    face_areas = []
    face_normals = []

    exp = TopExp_Explorer(solid, TopAbs_FACE)
    while exp.More():
        face = topods.Face(exp.Current())
        surf = BRepAdaptor_Surface(face)
        if surf.GetType() != GeomAbs_Plane:
            exp.Next()
            continue

        # Normal
        n = surf.Plane().Axis().Direction()
        n = np.array([n.X(), n.Y(), n.Z()], dtype=float)
        n /= np.linalg.norm(n)

        # Area
        gp = GProp_GProps()
        brepgprop.SurfaceProperties(face, gp)
        area = gp.Mass()

        face_normals.append(n)
        face_areas.append(area)
        exp.Next()

    # Sort OBB dimensions
    dims = sorted(obb_dims)  # [flange_thickness, web_depth, length]
    thickness, height, length = dims
    slenderness = length / height if height else 0
    aspect_ratio = height / thickness if thickness else 0

    # Area stats
    largest = max(face_areas) if face_areas else 0
    large_faces = [a for a in face_areas if a > 0.8 * largest]

    print("\n🔎 Shape fingerprint debug:")
    print(f"  OBB dimensions (sorted): {np.round(dims,1)}")
    print(f"  Slenderness ratio (L/H): {slenderness:.2f}")
    print(f"  Aspect ratio (H/t):      {aspect_ratio:.2f}")
    print(f"  Total planar faces:      {len(face_areas)}")
    print(f"  Large planar faces:      {len(large_faces)}")
    print(f"  Max area:                {max(face_areas):.1f}" if face_areas else "  Max area: —")
    print(f"  Large face areas:        {[round(a,1) for a in large_faces]}")
    
    # Fingerprint logic
    
    # --- Check for Beam ---
    if len(face_areas) >= 10 and slenderness > 4 and max(face_areas) > 100000:
        return "I"
    
    # --- Check for Angles ---
    elif len(large_faces) == 2 and slenderness > 6:
        a1, a2 = large_faces
        area_ratio = max(a1, a2) / min(a1, a2)
        if area_ratio < 1.1:
            return "L"
        else:
            return "L"

    # --- Check for Channel ---
    elif len(face_areas) >= 6 and slenderness > 6 and max(face_areas) > 50000:
        return "U"
        
    # --- Fallback ---
    return "Unknown"


In [5]:

# Display solid
display.DisplayShape(solid, update=True)

# Compute OBB
obb_data = compute_obb_and_local_axes(solid)

center       = obb_data['center']
xaxis        = obb_data['xaxis']
yaxis        = obb_data['yaxis']
zaxis        = obb_data['zaxis']
he_X         = obb_data['he_X']
he_Y         = obb_data['he_Y']
he_Z         = obb_data['he_Z']
half_extents = [he_X, he_Y, he_Z]
obb_dims     = [2*he_X, 2*he_Y, 2*he_Z]

# Draw bounding box
draw_obb(display,
         center=obb_data['center'],
         xaxis=obb_data['xaxis'],
         yaxis=obb_data['yaxis'],
         zaxis=obb_data['zaxis'],
         he_X=obb_data['he_X'],
         he_Y=obb_data['he_Y'],
         he_Z=obb_data['he_Z'],
         color="YELLOW")

# Identify profile type
profile_type = fingerprint_shape(shape, obb_dims)
print(f"\nProfile Type is : {profile_type}")

# Identify element - use classifier against library
section_face, section_origin = make_section_face_at_start(solid, center, zaxis, he_Z)
cs = prepare_classification_input(obb_data, profile_type, section_face)
matched_profile = classify_profile(cs, "../data/Shape_classifier_info.json")
print(f"\n🔍 Best match: {matched_profile['Designation']} in category {matched_profile['Category']}")
if "Swapped_dimensions" in matched_profile:
    print("⚠️  Matched after swapping width/height — OBB may be misaligned.")
    xaxis, yaxis = yaxis, xaxis
    he_X, he_Y = he_Y, he_X

display.FitAll()

ordered_corners = extract_ordered_section_corners(
    face=section_face,
    xaxis=xaxis,
    yaxis=yaxis,
    origin=section_origin
)

if profile_type == "L":
    should_flip = should_flip_zaxis_for_uea(
    corners_3d=ordered_corners,
    xaxis=xaxis,
    yaxis=yaxis,
    origin=section_origin,
    matched_profile=matched_profile
    )

    if should_flip:
        zaxis = zaxis.Reversed()
        print("🔄 Flipping Z axis to correct start orientation for angle profile")
    else:
        print("✅ Angle Z axis is correct")


🔎 Shape fingerprint debug:
  OBB dimensions (sorted): [  50.  100. 1000.]
  Slenderness ratio (L/H): 10.00
  Aspect ratio (H/t):      2.00
  Total planar faces:      8
  Large planar faces:      2
  Max area:                99921.5
  Large face areas:        [99921.5, 81921.5]

Profile Type is : L

🔍 Best match: 100x50x6 in category UEA
🔄 Angle Z axis is correct
